In [2]:
import numpy as np
import tvm
from tvm import te

In [5]:
n = te.var('n')
m = te.var('m')
A = te.placeholder((n, m), name='a')
B = te.compute((m, n), lambda i,j: A[j, i], name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True)

produce b {
  for (i, 0, m) {
    for (j, 0, n) {
      b[((i*stride) + (j*stride))] = a[((j*stride) + (i*stride))]
    }
  }
}

In [6]:
a = np.arange(12, dtype='float32').reshape(3, 4)
b = np.empty((4,3), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod = tvm.build(s, [A,B])
mod(a, b)
print(a)
print(b)

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]
[[ 0.  4.  8.]
 [ 1.  5.  9.]
 [ 2.  6. 10.]
 [ 3.  7. 11.]]


### Reshape

In [7]:
B = te.compute((m*n,), lambda i: A[i//m, i%m], name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A,B], simple_mode=True)

produce b {
  for (i, 0, (m*n)) {
    b[i] = a[((floordiv(i, m)*stride) + (floormod(i, m)*stride))]
  }
}

In [9]:
p, q = te.var('p'), te.var('q')
B = te.compute((p, q), lambda i,j: A[(i*q+j)//m,(i*q+j)%m], name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A,B], simple_mode=True)

produce b {
  for (i, 0, p) {
    for (j, 0, q) {
      b[((i*stride) + (j*stride))] = a[((floordiv(((i*q) + j), m)*stride) + (floormod(((i*q) + j), m)*stride))]
    }
  }
}

In [12]:
mod = tvm.build(s, [A,B])
a = np.arange(12, dtype='float32').reshape((3,4))
b = np.zeros((5,4), dtype='float32')
a, b = tvm.nd.array(a), tvm.nd.array(b)

mod(a, b)
print(b)

[[ 0.0000000e+00  1.0000000e+00  2.0000000e+00  3.0000000e+00]
 [ 4.0000000e+00  5.0000000e+00  6.0000000e+00  7.0000000e+00]
 [ 8.0000000e+00  9.0000000e+00  1.0000000e+01  1.1000000e+01]
 [-3.3655091e-37  3.0808948e-41 -3.0981601e-37  3.0808948e-41]
 [ 2.4074589e+27  4.5868703e-41 -3.3728918e-37  3.0808948e-41]]


### Slicing 

In [13]:
### b stands for begin, s stands for stride
bi, bj, si, sj = [te.var(name) for name in ['bi', 'bj', 'si', 'sj']]
B = te.compute(((n-bi)//si, (m-bj)//sj), lambda i,j: A[i*si+bi,j*sj + bj],name='b')
s = te.create_schedule(B.op)
mod = tvm.build(s, [A,B,bi,si,bj,sj])

In [16]:
b = tvm.nd.array(np.empty((1, 3), dtype='float32'))
mod(a, b, 1, 2, 1, 1)
np.testing.assert_equal(b.asnumpy(), a.asnumpy()[1::2, 1::1])
print(b.shape)

b = tvm.nd.array(np.empty((1, 2), dtype='float32'))
mod(a, b, 2, 1, 0, 2)
np.testing.assert_equal(b.asnumpy(), a.asnumpy()[2::1, 0::2])
print(b.shape)

(1, 3)
(1, 2)
